# **Trabalho Final - Inteligência Artificial - Classificação de Pokémon**

*Equipe:*
- Anna Beatriz - 538758
- Kauan Soares - 537063
- Leonan Marques - 539000
- Victoria Moura - 541801



# **Contextualização e Conceitos Utilizados**



A proposta principal do nosso trabalho é criar um modelo de I.A que seja capaz de identificar qual Pokemón aparece em cada imagem dentre um conjunto de imagens. Para isso, utilizamos uma abordagem baseada em Redes Neurais Convolucionais (CNNs), que são amplamente usadas para análise e reconhecimento de imagens.

No entanto, para que a CNN consiga aprender a distinguir cada Pokémon corretamente, faremos uso de aprendizado supervisionado. Isso significa que fornecemos ao modelo imagens já rotuladas, indicando a qual Pokémon cada uma pertence, e o modelo aprende, com base nesses exemplos, a fazer suas próprias previsões.

No nosso projeto, estruturamos a CNN com camadas especializadas:
-  Camadas *Convolucionais*, que identificam padrões locais nas imagens, como contornos e cores distintas.
- Camadas de *Pooling*, que reduzem a dimensionalidade da imagem, tornando o modelo mais eficiente.
- Camadas *Densas*, que combinam as informações extraídas e fazem a predição final do Pokémon.

Essa arquitetura permite que o modelo aprenda progressivamente a reconhecer características específicas de cada Pokémon e utilize essas informações para classificá-los corretamente.

# **Imports e inicialização**

In [ ]:
%pip install -r requirements.txt

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

# **Tratamento dos dados**





O dataset traz um código que ja divide as imagens em 80% treino, 10% validação e 10% teste

In [ ]:
# Definir diretórios dos conjuntos de imagens
train_dir = "./pokemon-dataset-1000 - Copia/train"
val_dir = "./pokemon-dataset-1000 - Copia/val"
test_dir = "./pokemon-dataset-1000 - Copia/test"

**`train_dir`**: Conjunto de imagens usado para treinar o modelo.

**`val_dir`**: Conjunto de imagens usado para validar o modelo durante o treinamento.

**`test_dir`**: Conjunto de imagens usado para avaliar o desempenho do modelo após o treinamento.

In [ ]:
# Parâmetros
img_size = (128, 128)
batch_size = 32

**`img_size`** : Define o tamanho de cada imagem que será alimentada ao modelo. Como o modelo de CNN está criando está lidando com imagens, é necessário garantir que todas as imagens sejam redimensionadas para um tamanho comum antes de serem passadas para a rede.  

**`batch_size`** : Define o número de imagens que serão passadas pela rede neural em uma única iteração durante o treinamento.  

In [ ]:
# Pré-processamento das imagens
train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

**OBS:** A normalização ajuda a acelerar o processo de treinamento, pois pode reduzir a variabilidade nos gradientes e melhorar a convergência. Também pode ajudar o modelo a se ajustar de maneira mais eficiente aos dados.

In [ ]:
# Criar os geradores de dados (Treino, Teste e Validação)
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size, class_mode="categorical"
)
val_generator = val_datagen.flow_from_directory(
    val_dir, target_size=img_size, batch_size=batch_size, class_mode="categorical"
)
test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=img_size, batch_size=batch_size, class_mode="categorical"
)

**`train_datagen:`** Este é o objeto de pré-processamento que foi configurado anteriormente (com a normalização dos pixels e, possivelmente, aumentos de dados, caso tenha sido configurado).  

**`flow_from_directory(train_dir):`** Este método cria um gerador que irá buscar as imagens no diretório train_dir (diretório que contém as imagens de treinamento). As imagens devem estar organizadas em subpastas, onde cada subpasta representa uma classe. O nome da subpasta será usado como o rótulo da classe.  

**`class_mode="categorical":`** Como estamos tratando um problema de classificação multiclasse, este parâmetro indica que as labels (rótulos) serão codificadas de forma categórica. Ou seja, para cada imagem, a saída será um vetor "one-hot" representando a classe da imagem.

In [ ]:
# Visualizar 5 exemplos do conjunto de treino
class_names = list(train_generator.class_indices.keys())
num_examples = 5

fig, axes = plt.subplots(1, num_examples, figsize=(15, 5))
train_images, train_labels = next(train_generator)

for i in range(num_examples):
    img = train_images[i]
    axes[i].imshow(img)
    axes[i].set_title(f"Classe: {class_names[np.argmax(train_labels[i])]} ")
    axes[i].axis("off")

plt.tight_layout()
plt.show()

A classe F1Score herda de tf.keras.metrics.Metric, o que permite criar métricas personalizadas para serem usadas com o TensorFlow/Keras.  
Definimos duas métricas auxiliares: Precision e Recall (essas duas métricas serão usadas para calcular o F1-Score.)  

**OBS**: `Precision` mede a proporção de predições positivas corretas, e `recall` mede a proporção de instâncias positivas que foram corretamente identificadas, ou seja, quantas das instâncias reais positivas foram capturadas pelo modelo.  

In [ ]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

O próximo método é chamado durante o treinamento e a avaliação do modelo para atualizar o estado da métrica.  
Ele recebe os valores verdadeiros (y_true) e preditos (y_pred), e, se necessário, os pesos amostrais (sample_weight). De modo que:

**`self.precision.update_state(y_true, y_pred, sample_weight):`** Atualiza o cálculo da precisão com base nos valores verdadeiros e preditos.

**`self.recall.update_state(y_true, y_pred, sample_weight):`** Atualiza o cálculo do recall com base nos valores verdadeiros e preditos.

In [ ]:
    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

Este método é chamado para calcular o valor da métrica (o F1-Score). Ele retorna o valor final da métrica com base nos valores de precisão e recall que foram atualizados. (A fórmula é uma média harmônica entre a precisão e o recall).

In [ ]:
    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

O próximo método é chamado para resetar o estado das métricas de precisão e recall. Isso é necessário entre as épocas de treinamento ou quando desejamos limpar as estatísticas internas para um novo cálculo.

In [ ]:
    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# **Rede Neural Convolucional - CNN**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

# Explicação do problema
# O objetivo deste modelo é classificar imagens de Pokémon utilizando uma Rede Neural Convolucional (CNN).
# O dataset contém múltiplas classes, cada uma representando um Pokémon diferente. O modelo será treinado,
# avaliado e testado para prever corretamente qual Pokémon está presente em cada imagem.


# Parâmetros
img_size = (128, 128)
batch_size = 32

# Pré-processamento das imagens
train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Criar os geradores de dados
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size, class_mode="categorical"
)
val_generator = val_datagen.flow_from_directory(
    val_dir, target_size=img_size, batch_size=batch_size, class_mode="categorical"
)
test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=img_size, batch_size=batch_size, class_mode="categorical"
)

# Definir a métrica F1 personalizada
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# Criar a CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

# Exibir arquitetura do modelo
model.summary()

# Compilar o modelo com as métricas de precisão, recall e F1Score
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy', Precision(), Recall(), F1Score()]
)

# Função para coletar as métricas durante o treinamento
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    verbose=1
)

# Avaliar o modelo no conjunto de teste
loss, accuracy, precision, recall, f1_score_value = model.evaluate(test_generator)
print(f'Loss no teste: {loss:.4f}, Acurácia no teste: {accuracy:.4f}, Precisão no teste: {precision:.4f}, Recall no teste: {recall:.4f}, F1-Score no teste: {f1_score_value:.4f}')

## 📌 Resumo das Camadas Iniciais da CNN

### 🎯 Objetivo:
Extrair características visuais

---

### 🔹 1. Camada: Convolução 2D
📌 **Função:**  
Detecta características espaciais nas imagens, detectando padrões locais.

✅ **Parâmetros:**  
| *Parâmetro*      | *Descrição* |
|--------------------|--------------|
| filters=32      | Define *32 filtros (ou kernels)* para detectar padrões na imagem (bordas, texturas, etc.). Cada filtro aprende uma característica diferente. |
| kernel_size=(3,3) | O tamanho do *filtro deslizante* é *3x3 pixels*. Isso significa que ele analisa pequenos blocos da imagem por vez. |
| activation='relu' | Usa a função de ativação *ReLU (Rectified Linear Unit)* para remover valores negativos e manter apenas os positivos. |
| input_shape=(128, 128, 3) | Especifica o formato das imagens de entrada: *128x128 pixels, com 3 canais de cor (RGB)*. |

**🎯 Saída:**
Matriz com 32 canais contendo as "versões filtradas" da imagem.

```python
Conv2D()
```

### 🔹 2. Camada: MaxPooling

📌 **Função:**
Redução das dimensões da imagem, reduzindo cálculos e melhorando a eficiência, mantendo apenas os pontos importantes.
Evita overfitting, pela redução da complexidade do modelo

```python
MaxPooling2D(pool_size=(2,2))
```

### 🔹 3. Camada: Segunda Convolução

📌 **Função:**
Como a imagem já foi reduzida antes pelo MaxPooling, agora os *filtros podem aprender padrões mais abstratos, como **formas completas (olhos, cauda, etc.)*.

✅ **Parâmetros**
| *Parâmetro*      | *Descrição* |
|--------------------|--------------|
| filters=64      | Agora a camada tem *64 filtros* para aprender *padrões mais complexos* (ex.: contornos e formatos). |
| kernel_size=(3,3) | Continua analisando pedaços de *3x3 pixels* da imagem. |
| activation='relu' | A função ReLU mantém apenas valores positivos. |


```python
Conv2D(filters=64, kernel_size=(3,3), activation='relu')
```

### 🔹 4. Camada: MaxPooling

📌 **Função:**
Como na sua utilização anterior, irá reduzir o tamanho da imagem pela metade

```python
MaxPooling2D(pool_size=(2,2))
```

### 🔹 5. Camada: Terceira Convolução

📌 **Função:**
Permitirá a rede reconhecer combinações de formas, no escopo do projeto, ela reconhecerá um pokémon inteiro, e não apenas seus contornos
Quando mais camadas convolucionais, mas poder de abstração o modelo terá

✅ **Parâmetros**
| *Parâmetro*      | *Descrição* |
|--------------------|--------------|
| filters=128      | Agora temos *128 filtros, pois estamos aprendendo padrões **ainda mais complexos*. |
| kernel_size=(3,3) | Mantemos um filtro pequeno para capturar detalhes. |
| activation='relu' | Continua removendo valores negativos. |

### 🔹 6. Camada: MaxPooling

📌 **Função:**
Reduz novamente o tamanho da imagem.
Agora a rede foca apenas nas *informações mais importantes*.

```python
MaxPooling2D(pool_size=(2,2))
```

## 📌 Resumo Geral das Camadas Iniciais
- Camada Convolucional extrai pequenos padrões da imagem.  
- MaxPooling reduz a dimensão e mantém apenas os pontos mais importantes.  
- Camada Convolucional aprende padrões mais abstratos.  
- MaxPooling reduz mais a dimensão.  
- Camada Convolucional aprende formas completas.  
- MaxPooling mantém apenas as informações mais relevantes.  


## 📌 Resumo das Camadas Finais da CNN (A partir de Flatten)

### 🎯 Objetivo:
Após extrair características visuais, as camadas finais **combinam essas informações para realizar a classificação**.

---

### 🔹 1. Camada **Flatten (Achatamento)**
📌 **Função:**  
Transforma os mapas de características (matrizes 2D) em um **vetor 1D** para ser processado pelas camadas densas.

✅ **Entrada:**  
Matriz de características extraídas pelas camadas convolucionais (ex.: `8x8x128`).

🎯 **Saída:**  
Vetor linear (`8192`, se `8*8*128`).

```python
Flatten()
```

### 🔹 2. Camada Densa (256 Neurônios, ReLU)

📌 **Função:**
Combina as características extraídas para reconhecer padrões abstratos e tomar decisões.

✅ **Parâmetros**

    Dense(256, activation='relu')
    256 neurônios aprendem padrões combinados.
    ReLU impede valores negativos e melhora o treinamento.

💡 **O que acontece aqui?**

    "Se há bordas pontiagudas e tom laranja predominante, é Charmander."
    "Se há orelhas arredondadas e tom amarelo, é Pikachu."

```python
Dense(256, activation='relu')
```

### 🔹 3. Camada Dropout (50%)

📌 **Função:**
Reduz overfitting desativando aleatoriamente 50% dos neurônios dessa camada a cada iteração.

✅ **Parâmetros**

    Dropout(0.5)

🎯 **Benefício:**

    ❌ Evita que o modelo fique muito dependente de neurônios específicos.
    ✅ Melhora generalização para novas imagens.

```python
Dropout(0.5)
```

### 🔹 4. Camada Densa Final (Saída, Softmax)

📌 **Função:**
Gera as probabilidades para cada classe de Pokémon.

✅ **Parâmetros**

    Dense(num_classes, activation='softmax')
    Número de neurônios = número de classes (ex.: 4 para Pikachu, Charmander, Bulbasaur e Squirtle).
    Softmax converte valores em probabilidades.

🎯 **Exemplo de saída:**

[Pikachu: 0.80, Charmander: 0.15, Bulbasaur: 0.03, Squirtle: 0.02]

➡ Resposta final: Pikachu! ✅

```python
Dense(num_classes, activation='softmax')
```

## 📌 Resumo Geral das Camadas
- Camada	Função Principal	Detalhes
- Flatten	Transforma mapas de características em um vetor 1D.	Entrada: matriz → Saída: vetor
- Dense (256, ReLU)	Aprende relações complexas entre padrões extraídos.	256 neurônios totalmente conectados.
- Dropout (50%)	Previne overfitting desativando neurônios aleatoriamente.	Evita que o modelo memorize o conjunto de treino.
- Dense (Softmax)	Gera as probabilidades de cada classe.	Número de neurônios = número de classes.

# **Visualização**

In [ ]:
# Mostrar algumas imagens de teste com rótulos reais e preditos
class_names = list(train_generator.class_indices.keys())
images, labels = next(test_generator)
predictions = model.predict(images)
pred_labels = np.argmax(predictions, axis=1)

num_images = min(len(images), 5)

fig, axes = plt.subplots(1, num_images, figsize=(10, 3))
for i in range(num_images):
    axes[i].imshow(images[i])
    true_label = class_names[np.argmax(labels[i])]
    predicted_label = class_names[pred_labels[i]]
    color = "green" if true_label == predicted_label else "red"
    axes[i].set_title(f"Real: {true_label}\nPred: {predicted_label}", color=color)
    axes[i].axis("off")
plt.show()

# Plotar as curvas das métricas
plt.figure(figsize=(15, 3))

# Acurácia
plt.subplot(1, 4, 1)
plt.plot(history.history['accuracy'], label='Acurácia Treino')
plt.plot(history.history['val_accuracy'], label='Acurácia Validação')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.title('Acurácia por Época')

# Precisão
plt.subplot(1, 4, 2)
plt.plot(history.history['precision'], label='Precisão Treino')
plt.plot(history.history['val_precision'], label='Precisão Validação')
plt.xlabel('Épocas')
plt.ylabel('Precisão')
plt.legend()
plt.title('Precisão por Época')

# Recall
plt.subplot(1, 4, 3)
plt.plot(history.history['recall'], label='Recall Treino')
plt.plot(history.history['val_recall'], label='Recall Validação')
plt.xlabel('Épocas')
plt.ylabel('Recall')
plt.legend()
plt.title('Recall por Época')

# F1-Score
plt.subplot(1, 4, 4)
plt.plot(history.history['f1_score'], label='F1-Score Treino')
plt.plot(history.history['val_f1_score'], label='F1-Score Validação')
plt.xlabel('Épocas')
plt.ylabel('F1-Score')
plt.legend()
plt.title('F1-Score por Época')

plt.tight_layout()
plt.show()

No contexto do treinamento de modelos de machine learning, especialmente em classificação, acurácia e precisão são métricas diferentes que avaliam diferentes aspectos do desempenho do modelo.

Acurácia: Proporção de previsões corretas (tanto verdadeiros positivos quanto verdadeiros negativos) em relação ao total de previsões feitas. É uma métrica geral que indica a porcentagem de previsões corretas feitas pelo modelo.

        Acurácia = (Verdadeiros positivos + Verdadeiros negativos) / Total de previsões


Precisão: A proporção de verdadeiros positivos em relação ao total de previsões positivas feitas pelo modelo. Mede a exatidão das previsões positivas do modelo. É particularmente útil quando o custo de falsos positivos é alto.

        Precisão = Verdadeiros Positivos / (Verdadeiros Positivos + Falsos Positivos)


Recall: É a proporção de verdadeiros positivos em relação ao total de positivos reais (verdadeiros positivos + falsos negativos). Mede a capacidade do modelo de identificar corretamente todas as instâncias positivas.

        Recall = Verdadeiros Positivos / (Verdadeiros Positivos + Falsos Negativos)

F1-Score: É a média harmônica da precisão e do recall. O F1-score é uma métrica útil quando se precisa equilibrar precisão e recall, especialmente em situações onde há uma distribuição desigual das classes.

        F1-Score = 2 x (Precisão x Recall) / (Precisão + Recall)


Isso permite uma avaliação completa do desempenho do modelo, considerando diferentes aspectos importantes para a tarefa de classificação.
